In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import pickle, json, sys, os, glob
import pandas as pd
pd.set_option('display.max_columns', None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import utils


from astropy.coordinates import SkyCoord
# from ctapipe.io import read_table
# from astropy.table import vstack, Table
# # from lstchain.io.io import dl1_params_lstcam_key, dl1_images_lstcam_key
# # from lstchain.io.io import dl1_params_tel_mon_ped_key, dl1_params_tel_mon_cal_key, dl1_params_tel_mon_flat_key
# import matplotlib.pyplot as plt
# import numpy as np

# from ctapipe.visualization import CameraDisplay
# from ctapipe.coordinates import EngineeringCameraFrame
# from ctapipe.instrument import SubarrayDescription
# from ctapipe.containers import EventType


from traitlets.config.loader import Config
from lstchain.io.config import get_standard_config

# Some configurations

In [2]:
# Source specifications
source_name = "crab"

# Maybe we add a way to extract the run numbers we are interested in
run_numbers = [6172, 6242, 6194, 6875, 6892, 6893]




# Getting coordinates of source
source_coords = SkyCoord.from_name(source_name)

# Paths to data and results

In [3]:
# Root path of this script
root = "/fefs/aswg/workspace/juan.jimenez/lst1_systematics/process_data/"
# Path to store the configuration file we are going to use
config_file = root + "objects/standard_config.json"
# Data main directory
root_data = root + f"../../data/cosmic_ray_data_correction/{source_name}/"

# STANDARD paths ---------
# DL1 data root
dl1_root = "/fefs/aswg/data/real/DL1/*/v*/tailcut84/"
# RFs root main directory
rfs_root = "/fefs/aswg/data/models/AllSky/20230901_v0.10.4_allsky_base_prod/"
# MCs dl2 main directory
mcs_root = "/fefs/aswg/data/mc/DL2/AllSky/20230901_v0.10.4_allsky_base_prod/TestingDataset/"


# Creating the directories in case they don't exist
for path in [config_file, root_data + "dl1b/", root_data + "dl2/", root_data + "dl3/", root_data + "irfs/"]:
    if not os.path.exists(path):
        os.makedirs(os.path.join(path), exist_ok=True)

# Opening and storing configuration file

In [4]:
config_dict = get_standard_config()
# print(config_dict)

#-------------------
# Changes in the configuration should be done here

# We select the heuristic flatfield option in the standard configuration
config_dict["source_config"]["LSTEventSource"]["use_flatfield_heuristic"] = True

#-------------------

with open(config_file, 'w') as json_file:
    json.dump(config_dict, json_file)

# Finding the files that interest us
#### Extracting dl1 files and dl1 datachecks

In [5]:
# We create a empty dictionary to store all the information needed inside
DICT = {}
for run in run_numbers:
    DICT[run] = {"run_num" : run}

DICT = utils.add_dl1_paths_to_dict(DICT, dl1_root)
DICT = utils.add_dl1_paths_to_dict(DICT, dl1_root, dchecking=True)


Adding dl1  data to dictionary...
...Finished adding dl1 data to dictionary

Adding dl1 datacheck data to dictionary...
...Finished adding dl1 data to dictionary


#### First the RFs we need to use for the source

#### Then the MC dl2 files associated to our observations

In [8]:
DICT[6875]

{'run_num': 6875,
 'dl1a_run': '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.h5',
 'dl1a_subrun': ['/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0000.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0001.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0002.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0003.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0004.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0005.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0006.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0007.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0008.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875.0009.h5',
  '/fefs/aswg/data/real/DL1/20220204/v0.9/tailcut84/dl1_LST-1.Run06875

In [ ]:
# source_ra  = f"{source_coords.ra.deg}deg"
# source_dec = f"{source_coords.dec.deg}deg"

In [ ]:
k = "source_config"
kk = "LSTEventSource"
for kkk in config[k][kk].keys():
    try:
        print(kkk, config[k][kk][kkk] == dictionary[k][kk][kkk])
    except:
        print(kkk, "ERRRO")

In [ ]:
k, kk, config[k][kk]

In [ ]:



# Reading the configuration file
config = Config(read_configuration_file(config_file))
config

In [ ]:
!lstchain_dl1ab -h

In [ ]:
test_dl1 = "test_data/dl1_LST-1.Run02977.0122.h5"
config_file = "/fefs/aswg/lstosa/config_files_lstchain/lstchain_standard_v0.10_heuristic_ff.json"
newname  = "test_data/dl1_LST-1.Run02977.0122_ls0.9.h5"

!lstchain_dl1ab \
    -f $test_dl1 \
    -o $newname \
    -c $config_file \
    --light-scaling 0.9 \

In [ ]:
subarray_info = SubarrayDescription.from_hdf(test_dl1)
camgeom = subarray_info.tel[1].camera.geometry

# Read parameters table from file:
dl1_parameters = read_table(test_dl1, dl1_params_lstcam_key)
dl1_parameters_b = read_table(newname, dl1_params_lstcam_key)
dl1_parameters.columns

In [ ]:
plt.hist2d(np.log10(dl1_parameters['intensity'][cosmics & valid_image]), np.log10(dl1_parameters_b['intensity'][cosmics & valid_image]), 200);

In [ ]:
cosmics = dl1_parameters['event_type'] == EventType.SUBARRAY.value
pedestals = dl1_parameters['event_type'] == EventType.SKY_PEDESTAL.value
flatfield = dl1_parameters['event_type'] == EventType.FLATFIELD.value

print('Cosmics (showers):', np.sum(cosmics))
print('Interleaved pedestals:', np.sum(pedestals))
print('Interleaved flatfield:', np.sum(flatfield))

In [ ]:
valid_image = dl1_parameters['intensity'] > 0 # Image survived image and was parametrized
print("Events which survived cleaning:")
print('Cosmics (showers):', np.sum(cosmics & valid_image))
print('Interleaved pedestals:', np.sum(pedestals & valid_image))
print('Interleaved flatfield:', np.sum(flatfield & valid_image))

In [ ]:
plt.figure(figsize=(10,4))
plt.hist(np.log10(dl1_parameters['intensity'][cosmics & valid_image]), 
         bins=120, range=(1, 7), histtype='step', log=True, label='cosmics')
plt.hist(np.log10(dl1_parameters['intensity'][flatfield & valid_image]), 
         bins=120, range=(1, 7), histtype='step', log=True, label='flatfield')
plt.hist(np.log10(dl1_parameters['intensity'][pedestals & valid_image]), 
         bins=120, range=(1, 7), histtype='step', log=True, label='pedestals')
plt.legend()
plt.xlabel('log10(Intensity/p.e.)')
plt.ylabel('events')
plt.show()

In [ ]:
dl1_images = read_table(test_dl1, dl1_images_lstcam_key)

# Locate first table entry for which n_pixels (after cleaning) is larger than 100, 
# is more or less centered (c.o.g. within 0.5 m of the camera center), and is a cosmic (="SUBARRAY"):
mask = ((dl1_parameters['intensity'] > 1000) &
        (dl1_parameters['n_pixels']>200) & 
        (dl1_parameters['r']<0.5*u.m) &
        (dl1_parameters['event_type']==EventType.SUBARRAY.value)
       )

index = np.where(mask)[0][0]

In [ ]:
plt.figure(figsize=(10,8))
camdisplay = CameraDisplay(camgeom, dl1_images['image'][index])
camdisplay.add_colorbar()
plt.show()